<a href="https://colab.research.google.com/github/laresamdeola/Machine-Learning/blob/main/Audio_Engineering_Model_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import numpy as np
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
import pandas as pd
from keras.layers import Dense, Dropout
from keras import regularizers
import matplotlib.pyplot as plt

In [187]:
import os

In [188]:
data_filepaths = os.listdir('/content/sample_data/Audio_Data')

In [189]:
len(data_filepaths)

496

In [190]:
data_filepaths[0:10]

['My best friend.m4a',
 'Escape plan.m4a',
 'Traffic.wav',
 'Dark skinned.wav',
 'In the road.wav',
 'Ready to make a move.wav',
 'Looking forward to winter.m4a',
 'Build.m4a',
 'After the storm.wav',
 'Rise.m4a']

In [191]:
os.getcwd()

'/content'

In [192]:
data_filepaths_2 = os.getcwd() + '/sample_data/Audio_Data'

In [193]:
data_filepaths_2

'/content/sample_data/Audio_Data'

In [194]:
os.getcwd()

'/content'

In [195]:
# filepaths

Function to extract features from audio files

In [196]:
def extract_features(filepaths, mfcc=True, chroma=False, mel=False):
  features = []
  for audio_file in filepaths:
      if mfcc:
        data, sample_rate = librosa.load(audio_file, sr=None)
        mfcc_features = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
        mfcc_mean = np.mean(mfcc_features.T, axis=0)
        # mfcc_mean[0:20]
        features.append(mfcc_mean)
      if chroma:
        chroma_features = librosa.feature.chroma_stft(y=audio_file, sr=sample_rate)
        chroma_features = np.mean(chroma_features.T,axis=0)
        features = np.hstack((features, chroma_features))
      if mel:
        mel_features = librosa.feature.melspectrogram(y=audio_file, sr=sample_rate)
        mel_features = np.mean(mel_features.T,axis=0)
        features = np.hstack((features, mel_features))
  return np.array(features)

In [197]:
# extract_features(data_file_paths)

Audio Files to ignore because of error

In [198]:
to_ignore = ['Region.MP4', 'Fight.m4a', 'Fear.MP4', 'Hurry.m4a', 'Ration.MP4', 'Decide.m4a', 'Bill.m4a', 'Race.MP4', 'Ready.MP4']

Methods to prepare dataset

In [199]:
def prepare_dataset(filepaths):
    features, labels = [], []
    augmented_features = []
    for file_path in filepaths:
        # Extract features
        if file_path in to_ignore:
          continue
        extracted_features = extract_features([file_path])
        features.append(extracted_features)
        # Assign label based on filename (assuming filename contains label)
        if 'm4a' in file_path:
          label = 'Noise'
        else:
          label = 'Truth'
        # label = file_path.split('/')[-1].split('-')[0]
        labels.append(label)
    return np.array(features), np.array(labels)

Prepare Dataset Visual Sample

In [200]:
# audio_frame = {
#     'X': [100, 10, 12],
#     'y': ['truth', 'truth', 'noise']
# }

# audio_frame = pd.DataFrame(audio_frame)

In [201]:
# audio_frame

Code to extract all the audio data

In [202]:
# data_file_paths = ['Achieve.MP4', 'Add.MP4']

all_file_paths = [os.getcwd() + '/sample_data/Audio_Data/' + i for i in data_filepaths]

In [203]:
X, y = prepare_dataset(all_file_paths)

<ipython-input-196-87758e6b0118>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-196-87758e6b0118>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-196-87758e6b0118>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  data, sample_rate = librosa.load(audio_file, sr=None)
/usr/loca

In [204]:
pd.DataFrame(y).value_counts()

Noise    249
Truth    247
Name: count, dtype: int64

Features/ Independent Variable

In [205]:
X

array([[[-2.6349460e+02,  1.3549956e+02,  9.4298035e-01, ...,
         -9.6149975e-01, -2.7571797e+00, -4.0854831e+00]],

       [[-5.8041412e+02,  1.2819296e+02,  2.1352489e+01, ...,
         -1.5119725e+00, -3.9320016e-01, -2.8538897e+00]],

       [[-4.8335712e+02,  1.6989563e+02,  3.0126531e+00, ...,
          7.5433534e-01,  4.5606175e-01, -2.2237989e-01]],

       ...,

       [[-2.7677820e+02,  1.4259013e+02,  4.5277343e+00, ...,
         -1.2313309e+00, -3.6060655e+00, -2.6067567e+00]],

       [[-2.4727470e+02,  1.3747842e+02,  1.1546179e+00, ...,
         -1.7641027e+00, -9.1363478e-01, -2.9849284e+00]],

       [[-4.9097961e+02,  1.0940868e+02,  7.8018475e+00, ...,
         -8.2052600e-01,  4.0874329e+00, -2.4583488e+00]]], dtype=float32)

Labels/Dependent/Target Variable

In [206]:
y

array(['Noise', 'Noise', 'Truth', 'Truth', 'Truth', 'Truth', 'Noise',
       'Noise', 'Truth', 'Noise', 'Truth', 'Noise', 'Truth', 'Noise',
       'Noise', 'Noise', 'Truth', 'Noise', 'Noise', 'Truth', 'Noise',
       'Noise', 'Noise', 'Truth', 'Truth', 'Noise', 'Truth', 'Noise',
       'Truth', 'Truth', 'Truth', 'Noise', 'Noise', 'Truth', 'Truth',
       'Truth', 'Noise', 'Noise', 'Noise', 'Noise', 'Noise', 'Noise',
       'Truth', 'Truth', 'Truth', 'Noise', 'Truth', 'Truth', 'Noise',
       'Truth', 'Truth', 'Noise', 'Noise', 'Noise', 'Noise', 'Noise',
       'Noise', 'Noise', 'Noise', 'Noise', 'Noise', 'Noise', 'Noise',
       'Noise', 'Truth', 'Noise', 'Truth', 'Noise', 'Truth', 'Truth',
       'Noise', 'Truth', 'Noise', 'Truth', 'Noise', 'Noise', 'Truth',
       'Noise', 'Truth', 'Truth', 'Truth', 'Noise', 'Truth', 'Truth',
       'Noise', 'Noise', 'Truth', 'Truth', 'Truth', 'Noise', 'Noise',
       'Noise', 'Truth', 'Noise', 'Truth', 'Truth', 'Truth', 'Noise',
       'Noise', 'Tru

Split the dataset into Train and Test set

In [207]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [208]:
X_train.ndim

3

Reshape the matrices into 2 dimension

In [209]:
X_train_flat = np.reshape(X_train, (X_train.shape[0], -1))
X_test_flat = np.reshape(X_test, (X_test.shape[0], -1))

In [210]:
X_train_flat.ndim

2

Convert string labels to binary labels

In [211]:
y_train_binary = np.where(y_train == "Truth", 1, 0)
y_test_binary = np.where(y_test == "Truth", 1, 0)

Define and train the deep learning model

In [212]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_flat.shape[1],)),
    Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    Dropout(0.5),
    tf.keras.layers.Dense(1, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [213]:
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train_flat.shape[1],), kernel_regularizer=regularizers.l2(0.01)),
#     tf.keras.layers.Dense(32, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])
# model.add(Dropout(0.5))
# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy', 'mse'])

In [214]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 128)               5248      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_28 (Dense)            (None, 32)                4128      
                                                                 
 dropout_3 (Dropout)         (None, 32)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 33        
                                                                 
Total params: 9409 (36.75 KB)
Trainable params: 9409 (36.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Fit and train the model

In [215]:
history = model.fit(X_train_flat, y_train_binary, epochs=50, batch_size=32, validation_split=0.2, validation_data=(X_test_flat, y_test_binary))

Epoch 1/50
12/12 [==============================] - 1s 18ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 2/50
12/12 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 3/50
12/12 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 4/50
12/12 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 5/50
12/12 [==============================] - 0s 5ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 6/50
12/12 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 7/50
12/12 [==============================] - 0s 6ms/step - loss: nan - accuracy: 0.4946 - val_loss: nan - val_accuracy: 0.5242
Epoch 8/50
12/12 [==============================] - 0s 6ms/st

Evaluation Metrics for ANN (Artificial Neural Network)

In [216]:
# from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# y_pred = model.predict(X_test_flat)

# # Assuming y_pred contains probabilities for each class
# predicted_labels = np.argmax(y_pred, axis=1)

# # Assuming y_pred contains binary predictions

# threshold = 0.998  # Adjust threshold as needed
# predicted_labels = np.where((y_pred >= threshold), 'Truth', 'Noise')
# predicted_labels = np.squeeze(predicted_labels)

# precision = precision_score(y_test, predicted_labels)
# recall = recall_score(y_test, predicted_labels)
# f1 = f1_score(y_test, predicted_labels)
# conf_matrix = confusion_matrix(y_test, predicted_labels)
# # print(predicted_labels.shape)
# # print(y_test.shape)
# print(precision)
# print(recall)
# print(f1)
# print(conf_matrix)

In [1]:
# # Plot training and validation loss
# plt.plot(history.history['loss'], label='Training Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [2]:
# # Plot training and validation accuracy
# plt.plot(history.history['accuracy'], label='Training Accuracy')
# plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.show()

Turn the binary target variable to either 'Noise' or 'Truth'

In [219]:
y_train_binary = np.where(y_train_binary == 0, 'Noise', 'Truth')

In [220]:
# y_test

In [221]:
# Assuming y_pred contains probabilities for each class

# predicted_labels = np.argmax(y_pred, axis=1)

In [222]:
# print(y_pred)

In [223]:
# print(y_test)

Using the Decision Tree

In [224]:
# Define and train the Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train_flat, y_train_binary)
y_pred_dt = clf.predict(X_test_flat)

In [225]:
# Calculate accuracy for Decision Tree classifier
accuracy = accuracy_score(y_test, y_pred_dt)
dt_confusion_matrix = confusion_matrix(y_test, y_pred_dt)
print("Decision Tree Accuracy:", accuracy)
print("Decision Tree Confusion Matrix:", dt_confusion_matrix)

Decision Tree Accuracy: 1.0
Decision Tree Confusion Matrix: [[65  0]
 [ 0 59]]


Using the Random Forest

In [226]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest Classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_flat, y_train_binary)
y_pred_rf = rf_classifier.predict(X_test_flat)
# rf_accuracy = accuracy_score(y_test, y_pred_rf)
# print("Random Forest Accuracy:", rf_accuracy)

In [227]:
# Calculate accuracy for Random Forest classifier
rf_accuracy = accuracy_score(y_test, y_pred_rf)
rf_confusion_matrix = confusion_matrix(y_test, y_pred_rf)
print("Random Forest Accuracy:", rf_accuracy)
print("Random Forest Confusion Matrix:", rf_confusion_matrix)

Random Forest Accuracy: 0.9919354838709677
Random Forest Confusion Matrix: [[64  1]
 [ 0 59]]


Using the Naive Bayes

In [228]:
from sklearn.naive_bayes import GaussianNB

# Naive Bayes Classifier
nb_classifier = GaussianNB()
nb_classifier.fit(X_train_flat, y_train_binary)
nb_predictions = nb_classifier.predict(X_test_flat)
nb_accuracy = accuracy_score(y_test, nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)

Naive Bayes Accuracy: 0.9758064516129032


In [229]:
from sklearn.metrics import classification_report
labels = ['Truth', 'Noise']
print(classification_report(y_test, nb_predictions, target_names=labels))

              precision    recall  f1-score   support

       Truth       0.97      0.98      0.98        65
       Noise       0.98      0.97      0.97        59

    accuracy                           0.98       124
   macro avg       0.98      0.98      0.98       124
weighted avg       0.98      0.98      0.98       124



Using the SVM - Support Vector Machines

In [230]:
from sklearn.svm import SVC

# Support Vector Machine (SVM) Classifier
svm_classifier = SVC()
svm_classifier.fit(X_train_flat, y_train_binary)
svm_predictions = svm_classifier.predict(X_test_flat)
svm_accuracy = accuracy_score(y_test, svm_predictions)
print("SVM Accuracy:", svm_accuracy)

SVM Accuracy: 0.8951612903225806


In [231]:
from sklearn.metrics import classification_report
labels = ['Truth', 'Noise']
print(classification_report(y_test, nb_predictions, target_names=labels))

              precision    recall  f1-score   support

       Truth       0.97      0.98      0.98        65
       Noise       0.98      0.97      0.97        59

    accuracy                           0.98       124
   macro avg       0.98      0.98      0.98       124
weighted avg       0.98      0.98      0.98       124



In [232]:
from sklearn.metrics import classification_report
labels = ['Truth', 'Noise']
print(classification_report(y_test, svm_predictions, target_names=labels))

              precision    recall  f1-score   support

       Truth       1.00      0.80      0.89        65
       Noise       0.82      1.00      0.90        59

    accuracy                           0.90       124
   macro avg       0.91      0.90      0.89       124
weighted avg       0.91      0.90      0.89       124



In [233]:
# Draw the ROC-AUC Curve
# Test with random data.